<a href="https://colab.research.google.com/github/jdmartinezrs/chatbotDevelopmentPython/blob/main/chatterbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Paso 1: Instalar ChatterBot (si no está instalado)
!pip install git+https://github.com/gunthercox/ChatterBot.git@master
!pip install chatterbot_corpus

In [ ]:
# Paso 2: Importar librerías necesarias
from chatterbot import ChatBot  # Clase base para crear el bot
from chatterbot.trainers import ListTrainer  # Entrenador manual

In [ ]:
# Paso 3: Crear el chatbot con configuración básica
# read_only=True → evita que el bot aprenda cosas incorrectas en tiempo real
# logic_adapters=['BestMatch'] → usa coincidencia de frases para responder lo más parecido posible
bot = ChatBot('TiendaBot', read_only=True, logic_adapters=['chatterbot.logic.BestMatch'])

In [ ]:
# Paso 4: Crear la lista de productos con sus precios
# Usamos un diccionario llamado 'productos' donde las claves son nombres y los valores sus precios
productos = {
    "arroz": 2000,
    "leche": 3000,
    "pan": 1000,
    "huevos": 5000
}
# Creamos una lista vacía llamada 'carrito' que almacenará lo que el usuario agregue
carrito = []

# Paso 5: Entrenar el bot con frases básicas
# Estas son frases predefinidas para que el bot sepa saludar, mostrar lista, agradecer y despedirse
conversacion = [
    "Hola", "¡Bienvenido a la tienda! ¿Qué deseas comprar?",
    "Lista de productos", "Tenemos: arroz ($2000), leche ($3000), pan ($1000), huevos ($5000)",
    "Gracias", "¡Con gusto!",
    "Adiós", "¡Hasta pronto!"
]

# Se entrena el bot con el listado anterior usando ListTrainer
trainer = ListTrainer(bot)
trainer.train(conversacion)

# Paso 6: Procesar la entrada del usuario de forma personalizada
# Esta función analiza la frase ingresada y responde con lógica personalizada según su contenido

def procesar_entrada(entrada):
    entrada = entrada.lower()  # Convertimos a minúsculas para facilitar comparaciones

    # Si el usuario menciona 'lista' o 'productos', mostrar el catálogo
    if "lista" in entrada or "productos" in entrada:
        return "Tenemos: " + ", ".join([f"{p} (${productos[p]})" for p in productos])

    # Si el usuario menciona un producto, lo añadimos al carrito
    elif any(prod in entrada for prod in productos):
        for prod in productos:
            if prod in entrada:
                carrito.append(prod)
                return f"{prod.capitalize()} añadido al carrito."

    # Si el usuario pregunta por el carrito, se muestra lo que hay
    elif "carrito" in entrada:
        if carrito:
            total = sum(productos[p] for p in carrito)    # Suma total con base en los productos añadidos
            return f"Tu carrito: {', '.join(carrito)}. Total: ${total}"
        else:
            return "Tu carrito está vacío."

    # Si el usuario quiere confirmar la compra
    elif "confirmar" in entrada or "comprar" in entrada:
        total = sum(productos[p] for p in carrito)
        carrito.clear()
        return f"Compra confirmada. Total: ${total}. ¡Gracias por tu compra!"

    # Si ninguna condición anterior se cumple, se deja que el bot responda
    else:
        return str(bot.get_response(entrada))

# Paso 7: Ejecutar el bucle conversacional
# Este bucle permite que el usuario escriba mensajes y reciba respuestas hasta que escriba 'salir'
print("Chatbot activado. Escribe 'salir' para terminar.")
while True:
    user_input = input("Tú: ")  # Entrada del usuario

    if user_input.lower() == "salir":
        print("Bot: ¡Gracias por visitarnos!")
        break  # Rompe el bucle y termina la conversación

    respuesta = procesar_entrada(user_input)  # Procesamos la entrada
    print("Bot:", respuesta)  # Mostramos la respuesta del bot